## Loading Libraries

In [1]:
import pandas as pd
import glob
import os
from sklearn.cluster import KMeans
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score
import numpy as np
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv(r"C:\Users\attafuro\Desktop\EV Charging Analysis\cleaned_traffic_data.csv")

In [4]:
df. head()

,Timestamp,Station,District,Route,Direction of Travel,Lane Type,Station Length,Samples,% Observed,Total Flow,...,Lane 5 Avg Speed,Lane 6 Flow,Lane 6 Avg Occ,Lane 6 Avg Speed,Lane 7 Flow,Lane 7 Avg Occ,Lane 7 Avg Speed,Lane 8 Flow,Lane 8 Avg Occ,Lane 8 Avg Speed
0,10/01/2024 00:00:00,308512,3,50,W,ML,3.995,197,0,497.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10/01/2024 00:00:00,311831,3,5,S,OR,NaN,101,92,27.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10/01/2024 00:00:00,311832,3,5,S,FR,NaN,101,92,78.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10/01/2024 00:00:00,311844,3,5,N,OR,NaN,202,92,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10/01/2024 00:00:00,311847,3,5,N,OR,NaN,303,92,73.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.shape

(4114680, 42)

## Basic Exploration 

In [6]:
# Define the final selected columns
selected_columns = [
    "Timestamp", "Station", "Route", "Direction of Travel",
    "Total Flow", "Avg Speed", "% Observed","Samples","Lane Type"
]

# Keep only the selected columns
df = df[selected_columns]

In [7]:
df.head()

,Timestamp,Station,Route,Direction of Travel,Total Flow,Avg Speed,% Observed,Samples,Lane Type
0,10/01/2024 00:00:00,308512,50,W,497.0,64.1,0,197,ML
1,10/01/2024 00:00:00,311831,5,S,27.0,NaN,92,101,OR
2,10/01/2024 00:00:00,311832,5,S,78.0,NaN,92,101,FR
3,10/01/2024 00:00:00,311844,5,N,43.0,NaN,92,202,OR
4,10/01/2024 00:00:00,311847,5,N,73.0,NaN,92,303,OR


In [8]:
df.isna().sum()

Timestamp                    0
Station                      0
Route                        0
Direction of Travel          0
Total Flow              303776
Avg Speed              1582594
% Observed                   0
Samples                      0
Lane Type                    0
dtype: int64

In [9]:
df.drop(columns=["Avg Speed"], inplace=True)

In [10]:
df.head()

,Timestamp,Station,Route,Direction of Travel,Total Flow,% Observed,Samples,Lane Type
0,10/01/2024 00:00:00,308512,50,W,497.0,0,197,ML
1,10/01/2024 00:00:00,311831,5,S,27.0,92,101,OR
2,10/01/2024 00:00:00,311832,5,S,78.0,92,101,FR
3,10/01/2024 00:00:00,311844,5,N,43.0,92,202,OR
4,10/01/2024 00:00:00,311847,5,N,73.0,92,303,OR


In [11]:
df.isna().sum()

Timestamp                   0
Station                     0
Route                       0
Direction of Travel         0
Total Flow             303776
% Observed                  0
Samples                     0
Lane Type                   0
dtype: int64

In [12]:
df = df[df['Total Flow'].notna()]

In [13]:
df.isna().sum()

Timestamp              0
Station                0
Route                  0
Direction of Travel    0
Total Flow             0
% Observed             0
Samples                0
Lane Type              0
dtype: int64

In [14]:
df.head()

,Timestamp,Station,Route,Direction of Travel,Total Flow,% Observed,Samples,Lane Type
0,10/01/2024 00:00:00,308512,50,W,497.0,0,197,ML
1,10/01/2024 00:00:00,311831,5,S,27.0,92,101,OR
2,10/01/2024 00:00:00,311832,5,S,78.0,92,101,FR
3,10/01/2024 00:00:00,311844,5,N,43.0,92,202,OR
4,10/01/2024 00:00:00,311847,5,N,73.0,92,303,OR


In [15]:
df.dtypes

Timestamp               object
Station                  int64
Route                    int64
Direction of Travel     object
Total Flow             float64
% Observed               int64
Samples                  int64
Lane Type               object
dtype: object

In [16]:
df.duplicated().sum()

0

In [17]:
df.reset_index().duplicated(subset=['Timestamp', 'Station']).sum()

0

In [18]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df.set_index('Timestamp', inplace=True)
df.sort_index(inplace=True)

In [19]:
# Convert Station to categorical type
df['Station'] = df['Station'].astype('category')

In [20]:
df.head()

,Station,Route,Direction of Travel,Total Flow,% Observed,Samples,Lane Type
Timestamp,,,,,,,
2024-10-01,308512,50,W,497.0,0,197,ML
2024-10-01,311831,5,S,27.0,92,101,OR
2024-10-01,311832,5,S,78.0,92,101,FR
2024-10-01,311844,5,N,43.0,92,202,OR
2024-10-01,311847,5,N,73.0,92,303,OR


## Feature Engineering

In [21]:
df['Hour'] = df.index.hour
df['Day'] = df.index.day
df['Weekday'] = df.index.weekday
df['IsWeekend'] = df['Weekday'].isin([5, 6]).astype(int)

In [22]:
for lag in range(1, 73):  # Lags 1 to 72
    df[f'TotalFlow_lag_{lag}'] = df['Total Flow'].shift(lag)

# Drop the rows that now have NaNs in these lag features (first 72 rows)
df.dropna(inplace=True)

In [23]:
df['Hour_sin'] = np.sin(2 * np.pi * df['Hour'] / 24)
df['Hour_cos'] = np.cos(2 * np.pi * df['Hour'] / 24)
df['Weekday_sin'] = np.sin(2 * np.pi * df['Weekday'] / 7)
df['Weekday_cos'] = np.cos(2 * np.pi * df['Weekday'] / 7)

In [24]:
df['Flow_roll_mean_6h'] = df['Total Flow'].rolling(window=6).mean()
df['Flow_roll_std_6h'] = df['Total Flow'].rolling(window=6).std()
df['Flow_roll_max_6h'] = df['Total Flow'].rolling(window=6).max()
df['Flow_roll_min_6h'] = df['Total Flow'].rolling(window=6).min()

In [25]:
for window in [3, 12, 24, 48, 72]:
    df[f'Flow_mean_{window}'] = df['Total Flow'].rolling(window=window).mean()
    df[f'Flow_std_{window}'] = df['Total Flow'].rolling(window=window).std()
    df[f'Flow_min_{window}'] = df['Total Flow'].rolling(window=window).min()
    df[f'Flow_max_{window}'] = df['Total Flow'].rolling(window=window).max()

In [26]:
df.dropna(inplace=True)

In [27]:
custom_holidays = [
    pd.to_datetime("2024-12-25"),
    pd.to_datetime("2024-12-31"),
    pd.to_datetime("2024-11-24"),
]
df['IsHoliday'] = df.index.to_series().apply(lambda x: 1 if x in custom_holidays else 0)

In [28]:
df.head()

,Station,Route,Direction of Travel,Total Flow,% Observed,Samples,Lane Type,Hour,Day,Weekday,...,Flow_max_24,Flow_mean_48,Flow_std_48,Flow_min_48,Flow_max_48,Flow_mean_72,Flow_std_72,Flow_min_72,Flow_max_72,IsHoliday
Timestamp,,,,,,,,,,,,,,,,,,,,,
2024-10-01,313344,50,W,190.0,92,202,ML,0,1,1,...,1068.0,395.833333,570.838627,3.0,3419.0,435.736111,522.596712,3.0,3419.0,0
2024-10-01,313349,50,W,190.0,92,202,ML,0,1,1,...,1068.0,399.458333,568.924516,3.0,3419.0,437.486111,521.544267,3.0,3419.0,0
2024-10-01,313368,80,W,787.0,100,719,ML,0,1,1,...,1068.0,404.375000,571.276906,3.0,3419.0,440.069444,522.824677,3.0,3419.0,0
2024-10-01,313386,80,W,1050.0,92,404,ML,0,1,1,...,1068.0,426.000000,575.736716,3.0,3419.0,443.555556,526.078366,3.0,3419.0,0
2024-10-01,313393,80,E,1064.0,92,505,ML,0,1,1,...,1068.0,376.937500,383.507686,3.0,1525.0,457.500000,529.066225,3.0,3419.0,0


In [29]:
for window in [12, 24, 48, 72]:
    df[f'Flow_CV_{window}'] = df[f'Flow_std_{window}'] / df[f'Flow_mean_{window}']

In [30]:
df.dropna(inplace=True)

In [31]:
# Group by station and calculate mean and std of Total Flow
station_stats = df.groupby('Station')['Total Flow'].agg(['mean', 'std'])
station_stats['CV'] = station_stats['std'] / station_stats['mean']
station_stats.dropna(inplace=True)  

In [32]:
df['IsHoliday'] = df['IsHoliday'].astype('category')
df['Route'] = df['Route'].astype('category')

In [33]:
#  Keep only numeric features
numeric_df = df.select_dtypes(include='number')

In [34]:
cat_features = [
    'Station', 'Route', 'Direction of Travel', 'Lane Type',
    'Weekday', 'IsWeekend', 'IsHoliday'
]

for col in cat_features:
    df[col] = df[col].astype('category')

In [35]:
# Define target and features
target = 'Total Flow'
features = [col for col in df.columns if col != target]

In [36]:
# Final feature groups
flow_features = [
    'Flow_mean_3', 'Flow_std_3', 'Flow_max_3', 'Flow_min_3',
    'Flow_mean_72', 'Flow_std_72', 'Flow_std_48', 'Flow_max_24',
    'Flow_mean_24', 'Flow_mean_48',
    'Flow_roll_std_6h', 'Flow_roll_max_6h',
    'Flow_CV_48'
]

lag_features = [
    'TotalFlow_lag_1', 'TotalFlow_lag_2', 'TotalFlow_lag_3',
    'TotalFlow_lag_7', 'TotalFlow_lag_8',
    'TotalFlow_lag_15', 'TotalFlow_lag_18'
]

time_features = [
    'Hour', 'Hour_cos', 'Hour_sin', 'Weekday_cos'
]

quality_features = [
    'Samples', '% Observed'
]

# Copy original DataFrame
df_model = df.copy()

# One-hot encode low-cardinality features
df_model = pd.get_dummies(df_model, columns=['IsHoliday', 'Direction of Travel'], drop_first=True)

# Label encode high-cardinality categorical features
for col in ['Station', 'Route', 'Lane Type']:
    le = LabelEncoder()
    df_model[col] = le.fit_transform(df_model[col])

# Capture dummy columns dynamically
dummy_columns = [col for col in df_model.columns if col.startswith('IsHoliday_') or col.startswith('Direction of Travel_')]

# Final feature list
categorical_features = ['Lane Type', 'Route', 'Station']
selected_features = flow_features + lag_features + time_features + quality_features + categorical_features + dummy_columns


In [37]:
df_model.head()

,Station,Route,Total Flow,% Observed,Samples,Lane Type,Hour,Day,Weekday,IsWeekend,...,Flow_min_72,Flow_max_72,Flow_CV_12,Flow_CV_24,Flow_CV_48,Flow_CV_72,IsHoliday_1,Direction of Travel_N,Direction of Travel_S,Direction of Travel_W
Timestamp,,,,,,,,,,,,,,,,,,,,,
2024-10-01,144,7,190.0,92,202,3,0,1,1,0,...,3.0,3419.0,0.581854,0.884330,1.442119,1.199342,False,False,False,True
2024-10-01,145,7,190.0,92,202,3,0,1,1,0,...,3.0,3419.0,0.648914,0.895621,1.424240,1.192139,False,False,False,True
2024-10-01,146,11,787.0,100,719,3,0,1,1,0,...,3.0,3419.0,0.534337,0.827111,1.412740,1.188050,False,False,False,True
2024-10-01,147,11,1050.0,92,404,3,0,1,1,0,...,3.0,3419.0,0.530732,0.817180,1.351495,1.186048,False,False,False,True
2024-10-01,148,11,1064.0,92,505,3,0,1,1,0,...,3.0,3419.0,0.545745,0.772216,1.017430,1.156429,False,False,False,False


In [38]:
import numpy as np
import torch

# Pivot Total Flow into [T, N] format
flow_matrix = df_model.pivot_table(index=df_model.index, columns='Station', values='Total Flow')
flow_matrix = flow_matrix.sort_index()
flow_matrix = flow_matrix.fillna(method='ffill').fillna(0)  # Handle any last NaNs

# Create adjacency matrix using Pearson correlation
correlation_matrix = flow_matrix.corr().values  # shape: (N, N)
threshold = 0.5  # you can tune this
adj_matrix = (correlation_matrix >= threshold).astype(float)

# Normalize adjacency matrix: A_hat = D^-0.5 * (A + I) * D^-0.5
A = adj_matrix + np.eye(adj_matrix.shape[0])  # add self-loops
D = np.diag(1.0 / np.sqrt(np.sum(A, axis=1)))
A_norm = D @ A @ D  # symmetric normalization
A_norm = torch.tensor(A_norm, dtype=torch.float32)

print("Flow matrix shape (T, N):", flow_matrix.shape)
print("Adjacency matrix shape:", A_norm.shape)


Flow matrix shape (T, N): (2208, 1806)
Adjacency matrix shape: torch.Size([1806, 1806])


In [39]:
# We only have 1 feature (Total Flow), so reshape
flow_tensor = torch.tensor(flow_matrix.values, dtype=torch.float32).unsqueeze(-1)  # shape: (T, N, 1)
print("Final data tensor shape (T, N, F):", flow_tensor.shape)


Final data tensor shape (T, N, F): torch.Size([2208, 1806, 1])


In [40]:
def create_X_Y_pairs(data_tensor, lag=24, horizon=72):
    """
    Converts full data tensor into input/output sequences.
    Args:
        data_tensor: (T, N, F)
        lag: number of time steps in input sequence
        horizon: number of future steps to predict
    Returns:
        X: (num_samples, lag, N, F)
        Y: (num_samples, N, horizon)
    """
    X, Y = [], []
    T = data_tensor.shape[0]
    for t in range(lag, T - horizon):
        x_seq = data_tensor[t - lag:t]                   # (lag, N, F)
        y_seq = data_tensor[t:t + horizon, :, 0].T       # (N, horizon)
        X.append(x_seq)
        Y.append(y_seq)
    return torch.stack(X), torch.stack(Y)


X, Y = create_X_Y_pairs(flow_tensor, lag=24, horizon=72)
print("X shape:", X.shape)  # (samples, 24, N, 1)
print("Y shape:", Y.shape)  # (samples, N, 72)


X shape: torch.Size([2112, 24, 1806, 1])
Y shape: torch.Size([2112, 1806, 72])


In [41]:
from torch.utils.data import TensorDataset, DataLoader

# Split
num_samples = X.shape[0]
train_len = int(0.6 * num_samples)
val_len   = int(0.2 * num_samples)
test_len  = num_samples - train_len - val_len

X_train, Y_train = X[:train_len], Y[:train_len]
X_val,   Y_val   = X[train_len:train_len + val_len], Y[train_len:train_len + val_len]
X_test,  Y_test  = X[train_len + val_len:], Y[train_len + val_len:]

# Dataloaders
batch_size = 64
train_loader = DataLoader(TensorDataset(X_train, Y_train), batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(TensorDataset(X_val, Y_val), batch_size=batch_size)
test_loader  = DataLoader(TensorDataset(X_test, Y_test), batch_size=batch_size)


In [42]:
import torch.nn as nn

class STGCN_GRU_LSTM(nn.Module):
    def __init__(self, num_nodes, in_features, gcn_out, rnn_hidden, horizon, rnn_type='gru', A_norm=None):
        super(STGCN_GRU_LSTM, self).__init__()
        self.num_nodes = num_nodes
        self.rnn_type = rnn_type.lower()
        self.horizon = horizon
        
        # Register adjacency matrix
        self.register_buffer('A_norm', A_norm)  # (N, N)

        # Graph convolution weights
        self.gcn_weight = nn.Parameter(torch.randn(in_features, gcn_out))
        self.gcn_bias   = nn.Parameter(torch.zeros(gcn_out))

        # RNN: GRU or LSTM
        if self.rnn_type == 'gru':
            self.rnn = nn.GRU(input_size=gcn_out, hidden_size=rnn_hidden, batch_first=True)
        else:
            self.rnn = nn.LSTM(input_size=gcn_out, hidden_size=rnn_hidden, batch_first=True)

        # Output layer: forecast next `horizon` steps
        self.fc = nn.Linear(rnn_hidden, horizon)

    def forward(self, x):
        # x: (B, T, N, F)
        B, T, N, F = x.shape

        # Spatial Graph Convolution: apply A_norm at each time step
        x = x.view(B * T, N, F)                          # (B*T, N, F)
        A_exp = self.A_norm.unsqueeze(0).expand(B * T, -1, -1)  # (B*T, N, N)
        x = torch.bmm(A_exp, x)                         # (B*T, N, F)
        x = torch.matmul(x, self.gcn_weight) + self.gcn_bias   # (B*T, N, gcn_out)
        x = torch.relu(x)
        x = x.view(B, T, N, -1)                          # (B, T, N, gcn_out)

        # Temporal Modeling: each node becomes a sequence
        x = x.permute(0, 2, 1, 3)                        # (B, N, T, gcn_out)
        x = x.reshape(B * N, T, -1)                      # (B*N, T, gcn_out)

        if self.rnn_type == 'gru':
            rnn_out, _ = self.rnn(x)
        else:
            rnn_out, (_, _) = self.rnn(x)

        last_output = rnn_out[:, -1, :]                  # (B*N, rnn_hidden)
        out = self.fc(last_output)                       # (B*N, horizon)
        out = out.view(B, N, self.horizon)               # (B, N, horizon)

        return out


In [43]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = STGCN_GRU_LSTM(
    num_nodes = X.shape[2],
    in_features = X.shape[3],
    gcn_out = 16,
    rnn_hidden = 32,
    horizon = 72,
    rnn_type = 'gru',         # or 'lstm'
    A_norm = A_norm
).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print(" Model initialized on", device)


 Model initialized on cpu


In [44]:
num_epochs = 50
best_val_loss = float('inf')

for epoch in range(1, num_epochs + 1):
    model.train()
    train_loss = 0.0

    for X_batch, Y_batch in train_loader:
        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
        optimizer.zero_grad()
        Y_pred = model(X_batch)  # (B, N, horizon)
        loss = criterion(Y_pred, Y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * X_batch.size(0)

    train_loss /= len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_val_batch, Y_val_batch in val_loader:
            X_val_batch, Y_val_batch = X_val_batch.to(device), Y_val_batch.to(device)
            Y_val_pred = model(X_val_batch)
            val_loss += criterion(Y_val_pred, Y_val_batch).item() * X_val_batch.size(0)

    val_loss /= len(val_loader.dataset)
    
    print(f"Epoch {epoch:02d}: Train MSE = {train_loss:.4f} | Val MSE = {val_loss:.4f}")


Epoch 01: Train MSE = 2687752.7735 | Val MSE = 2591791.4834
Epoch 02: Train MSE = 2686835.8721 | Val MSE = 2590708.1090
Epoch 03: Train MSE = 2685484.5742 | Val MSE = 2589270.9408
Epoch 04: Train MSE = 2684038.2839 | Val MSE = 2587725.6327
Epoch 05: Train MSE = 2682197.3747 | Val MSE = 2585867.8306
Epoch 06: Train MSE = 2680504.6648 | Val MSE = 2584288.0616
Epoch 07: Train MSE = 2678947.8240 | Val MSE = 2582819.6374
Epoch 08: Train MSE = 2677472.3838 | Val MSE = 2581404.9834
Epoch 09: Train MSE = 2676043.2824 | Val MSE = 2580025.5486
Epoch 10: Train MSE = 2674644.4976 | Val MSE = 2578670.6564
Epoch 11: Train MSE = 2673266.9718 | Val MSE = 2577334.4621
Epoch 12: Train MSE = 2671906.3268 | Val MSE = 2576012.0237
Epoch 13: Train MSE = 2670557.9588 | Val MSE = 2574700.8957
Epoch 14: Train MSE = 2669220.6119 | Val MSE = 2573398.6232
Epoch 15: Train MSE = 2667891.8433 | Val MSE = 2572104.2370
Epoch 16: Train MSE = 2666570.4872 | Val MSE = 2570816.9941
Epoch 17: Train MSE = 2665255.5923 | Val

In [45]:
model.eval()
Y_preds, Y_actuals = [], []

with torch.no_grad():
    for X_batch, Y_batch in test_loader:
        X_batch = X_batch.to(device)
        Y_batch = Y_batch.to(device)
        Y_pred = model(X_batch)
        Y_preds.append(Y_pred.cpu())
        Y_actuals.append(Y_batch.cpu())

Y_preds = torch.cat(Y_preds, dim=0)       # (samples, N, horizon)
Y_actuals = torch.cat(Y_actuals, dim=0)


In [46]:
import numpy as np

Y_pred_np = Y_preds.numpy()
Y_actual_np = Y_actuals.numpy()
horizons = [12, 24, 48, 72]

print("\n Horizon-wise Evaluation:")
for h in horizons:
    idx = h - 1  # 0-based index
    pred_h = Y_pred_np[:, :, idx]
    actual_h = Y_actual_np[:, :, idx]
    err = pred_h - actual_h
    mae = np.mean(np.abs(err))
    rmse = np.sqrt(np.mean(err ** 2))
    print(f"Horizon {h:>2d} hours → MAE: {mae:.4f} | RMSE: {rmse:.4f}")



 Horizon-wise Evaluation:
Horizon 12 hours → MAE: 884.3862 | RMSE: 1533.3093
Horizon 24 hours → MAE: 897.2598 | RMSE: 1545.3198
Horizon 48 hours → MAE: 897.5784 | RMSE: 1544.2327
Horizon 72 hours → MAE: 895.2085 | RMSE: 1539.2424
